In [14]:
import pyscf
from pyscf import dft
from pyscf import gto
from pyscf import lib
from pyscf.dft import libxc
from pyscf.dft import numint
from pyscf.dft import xc_deriv
import ctypes
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


In [16]:
mol = gto.Mole()
mol.atom='''
    H 0 0 -1
    H 0 0 1
'''
mol.basis = 'cc-pvdz'
mol.symmetry = True
mol.build()
#spin = 0
pbe_e = dft.RKS(mol)
pbe_e.xc = 'pbe,pbe' #xc basis
pbe_e.kernel()

converged SCF energy = -0.995650548182693


np.float64(-0.9956505481826932)

In [56]:
grid = dft.gen_grid.Grids(mol)
grid.level = 7
grid.build()



In [58]:
weights = grid.weights
coords = grid.coords
#get phi values and gradients from grid using numint method
ao_value_grid = numint.eval_ao(mol, coords, deriv=1)
print(ao_value_grid.shape)
print("Phi Matrix Dimensions:", ao_value_grid[0].shape)

(4, 100344, 10)
Phi Matrix Dimensions: (100344, 10)


In [60]:
phi = ao_value_grid[0]
phix = ao_value_grid[1]
phiy = ao_value_grid[2]
phiz = ao_value_grid[3]

dm = pbe_e.make_rdm1()
rho_grid = numint.eval_rho(mol, ao_value_grid, dm, xctype='GGA')
print(rho_grid.shape)
print(dm.shape)

(4, 100344)
(10, 10)


In [123]:
#can compute Exc as a group, or by parts (Ex, Ec) using numint method
_itrf = lib.load_library('libxc_itrf')
def evalexc(rho): #only density needed as variable
    ni = numint.NumInt()
    exc = ni.eval_xc("pbe,pbe", rho, spin=0, relativity=0, deriv=1)
    #calc totals
    Exc = np.einsum('p,p,p->', exc[0], rho[0], weights)
    vxc_list = exc[1] #vxc (restricted) = (vxcrho, vxcsigma, vlapl, vtau)
    Vxctmp = 0.5*np.einsum('pb,p,p,pa->ab', phi, vxc_list[0], weights, phi, optimize=True) #vxcrho
    Vxctmp += 2.0*np.einsum('pb,p,p,p,pa->ab', phix, vxc_list[1], rho[1], weights, phi, optimize = True) #vxcsigma
    Vxctmp += 2.0*np.einsum('pb,p,p,p,pa->ab', phiy, vxc_list[1], rho[2], weights, phi, optimize = True) #vxcsigma
    Vxctmp += 2.0*np.einsum('pb,p,p,p,pa->ab', phiz, vxc_list[1], rho[3], weights, phi, optimize = True) #vxcsigma
    Vxc = Vxctmp+Vxctmp.T
    return Exc, Vxc


In [125]:
dft.libxc.xc_reference('GGA_X_PBE')

['J. P. Perdew, K. Burke, and M. Ernzerhof.,  Phys. Rev. Lett. 77, 3865 (1996)',
 'J. P. Perdew, K. Burke, and M. Ernzerhof.,  Phys. Rev. Lett. 78, 1396 (1997)']

In [94]:
#test function against kernel values
print("Exchange correlation energy:", pbe_e.get_veff().exc)
test_e, test_v = evalexc(rho_grid)
print(test_e)
print(pbe_e.get_veff().exc - test_e)

Exchange correlation energy: -0.5108681474766017
-0.5108684514165607
3.039399589788161e-07


In [97]:
#Set up SCF loop (define the variables)
ndocc = mol.nelectron // 2 #number of occupied orbitals, each with 2 electrons
I = mol.intor('int2e') #ERI Tensor
S = mol.intor('int1e_ovlp') #Overlap matrix
e_val, e_vec = np.linalg.eigh(S) #returns the eigenvalues[0] and eigenvectors[1] of a matrix
A = e_vec @ np.diag(e_val**-0.5) @ e_vec.T

T = mol.intor('int1e_kin')
V = mol.intor('int1e_nuc') 
H = T+V

#Fock matrix
F_p = A.dot(H).dot(A)
e, C_p = np.linalg.eigh(F_p) #eigenvalues = e, eigenvectors = C_p
C = A.dot(C_p) #transforms the eigenvectors back to AO basis
C_occ = C[:, :ndocc] #occupied orbitals, used for density matrix
D =2* np.einsum('pi,qi->pq', C_occ, C_occ, optimize=True) #times 2 bc 2 e- per orbital
print(D.shape)

(10, 10)


In [105]:
#SCF Loop iterations
E_old = 0.0
MAXITER = 40
E_conv = 1.00e-8
D_conv = 1.00e-6
F_list = []
DIIS_RESID = []

for i in range(1, MAXITER+1):
    J = np.einsum('pqrs,rs->pq', I, D, optimize=True)
    rho_grid = numint.eval_rho(mol, ao_value_grid, D, xctype="GGA")
    Exc, Vxc = evalexc(rho_grid)
    F = H + J + Vxc
    F_list.append(F)
    
    #Components of the total SCF energy
    H_E = np.sum(D * H)
    J_E = 0.5*np.sum(D*J)
    
    #DIIS resid
    diis_r = A.dot(F.dot(D).dot(S) - S.dot(D).dot(F)).dot(A)
    DIIS_RESID.append(diis_r)
    #print(D)
    #SCF energy
    SCF_E = H_E + J_E + Exc + mol.energy_nuc() #total energy
    #Check difference
    diff_e = SCF_E - E_old
    dRMS = np.mean(diis_r**2)**0.5
    #Count iterations and keep track of values
    print("SCF Iteration %3d: SCF Energy = %4.16f, diff_E = %1.5E, dRMS = %1.5E" % (i, SCF_E, diff_e, dRMS))
    #create break sequence
    if (abs(diff_e) < E_conv) and (dRMS < D_conv):
        break
    #Redefine variables for next iteration
    E_old = SCF_E
    #Update DIIS for better convergence - B matrix 
    if i >=2:
        B_dim = len(F_list)+1
        B = np.empty((B_dim, B_dim))
        B[-1,:] = -1
        B[:,-1] = -1
        B[-1, -1] = 0
        for j in range(len(F_list)):
            for k in range(len(F_list)):
                B[j, k] = np.einsum('jk,jk->', DIIS_RESID[j], DIIS_RESID[k], optimize=True)
        #Pulay Equation
        rhs = np.zeros((B_dim))
        rhs[-1] = -1
        coeff = np.linalg.solve(B, rhs) # solve pulay w/ numpy
        F = np.zeros_like(F) #DIIS fock matrix
        
        for p in range(coeff.shape[0]-1):
            F += coeff[p] * F_list[p]
            
    F_p = A.dot(F).dot(A) #new orbital guess, using DIIS F matrix
    e, C_p = np.linalg.eigh(F_p)
    C = A.dot(C_p)
    C_occ = C[:, :ndocc]
    
    D = 2*np.einsum('pi,qi->pq', C_occ, C_occ, optimize=True)
    #D matrix improved upon for next iteration
    #Now failsafe in case doesn't converge
    if (i==MAXITER):
        raise Exception("Too many iterations before Convergence")
print("------")
print(f"SCF Converged, final RHF Energy: {SCF_E:.13f} [Eh]")
print(f'Coulomb Energy: {J_E} Hartree')
print(f'Exchange-correlation energy: {Exc} Hartree')
print(f'Nuclear repulsion energy: {mol.energy_nuc()} Hartree')

SCF Iteration   1: SCF Energy = -0.9956508521228675, diff_E = -9.95651E-01, dRMS = 9.65196E-12
SCF Iteration   2: SCF Energy = -0.9956508521228662, diff_E = 1.33227E-15, dRMS = 5.23419E-11
------
SCF Converged, final RHF Energy: -0.9956508521229 [Eh]
Coulomb Energy: 0.8628348373718194 Hartree
Exchange-correlation energy: -0.5108686460934776 Hartree
Nuclear repulsion energy: 0.26458860546 Hartree


In [109]:
#model calculation
model = dft.RKS(mol)
model.xc = 'pbe,pbe'
model.kernel()
print('Coulomb energy:', model.get_veff().ecoul, 'Hartree')
print('Exchange-correlation model:', model.get_veff().exc, 'Hartree')
print('Nuclear repulsion model:', model.energy_nuc(), 'Hartree')
print('Electronic Energy model:', model.energy_elec()[0], 'Hartree') #(Eelec, 2e- contribution)

converged SCF energy = -0.995650548182693
Coulomb energy: 0.8628345777106594 Hartree
Exchange-correlation model: -0.5108681474766017 Hartree
Nuclear repulsion model: 0.26458860546 Hartree
Electronic Energy model: -1.2602391536426931 Hartree
